### Minicurso LABREN explorando dados com IA 🚀
Neste minicurso, você aprenderá a explorar conjuntos de dados utilizando a ajuda de um assistente de IA. Usaremos um contjunto de dados metorológicos para analisar suas variáveis e gerar visualizações que facilitam a interpretação dos resultados.

- Dados Meteorológicos: dados_meteorologicos.parquet


#### Objetivos do Minicurso:

- 🤖 Formular prompts para que a IA auxilie na criação de códigos.
- 🗄️ Integrar SQL com o DuckDB para carregar e manipular dados em formato Parquet.
- 📊 Explorar os dados por meio de consultas SQL para obter insights relevantes.
- 📈 Gerar visualizações (inclusive espaciais) que facilitam a interpretação dos resultados, aproveitando as coordenadas presentes nos dados.

#### Cronograma:

1. Introdução e Objetivos:

- Apresentação do minicurso e dos conjuntos de dados.
- Explicação do papel da IA na criação de código e análise de dados.

2. Configuração do Ambiente:

- Importar bibliotecas para abrir dados em formato Parquet, manipulação e visualização, incluindo visualizações espaciais.

3. Carregamento dos Dados:

- Criar comandos para carregar os dados Parquet.
- Visualizar os 5 primeiros registros do conjunto de dados.

4. Exploração dos Dados:

- Realizar consultas SQL para explorar as variáveis do conjunto de dados.
- Gerar visualizações para facilitar a interpretação dos resultados.

5. Conclusão:

### 1. Configuração do Ambiente ⚙️

Para começar, vamos importar as bibliotecas necessárias para abrir os dados em formato Parquet, manipulação e visualização, incluindo visualizações espaciais.

In [ ]:
# Instale, importe e inicialize bibliotecas para abrir dados no formato Parquet usando DuckDB
# e também bibliotecas para visualização dos dados.
!pip install duckdb pandas matplotlib seaborn

import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Inicialize o DuckDB
con = duckdb.connect(database=':memory:', read_only=False)

#### 2. Download e Carregamento dos Dados 📊

Baixe os dados meteorológicos para o ambiente de execução do minicurso. Em seguida, carregue os dados para visualizar os 5 primeiros registros do conjunto de dados.

In [ ]:
# Baixe o os arquivos dados_meteorologicos.parquet que estão no google drive
#  ulr vou listar para você:

url_meteoro = 'https://drive.google.com/uc?id=1MES_nefJ_V2ZamOFTXzzHCdtH7Fh0qq2'

# Instale a biblioteca gdown para baixar os arquivos
!pip install gdown

# Baixe os arquivos
import gdown

gdown.download(url_meteoro, 'dados_meteorologicos.parquet', quiet=False)

In [ ]:
# Crie um código para carregar os dados dos arquivos dados_meteorologicos.parquet utilizando o DuckDB.
# Mostre os 5 primeiros registros de cada conjunto como um DataFrame do Pandas.
ARQV_METEOROLOGICO = 'dados_meteorologicos.parquet'

# Crie uma query para gerar uma tabela no DuckDB com os dados meteorológicos caso ela não exista
query = f'CREATE TABLE IF NOT EXISTS dados_meteorologicos AS SELECT * FROM parquet_scan(\'{ARQV_METEOROLOGICO}\')'

# Execute a query
con.execute(query)

In [ ]:
# Mostre os 5 primeiros registros da tabela
query = 'SELECT * FROM dados_meteorologicos LIMIT 5'

# Execute a query
df = con.execute(query).fetchdf()
df

In [ ]:
# Crie um código para mostrar a quantidade de registros de cada conjunto de dados.
# Mostre a quantidade de registros de cada estação meteorológica (coluna "acronym").
query = 'SELECT acronym,' \
        '       COUNT(*) AS quantidade_registros' \
        '  FROM dados_meteorologicos' \
        ' GROUP BY acronym' \
        ' ORDER BY quantidade_registros DESC'

# Execute a query
df = con.execute(query).fetchdf()
df

### 4. Visualizações dos Dados 📊

Vamos criar gráficos de linha e barras para visualizar as informações extraídas. 

In [ ]:
# Plote um gráfico de barras para mostrar a quantidade de registros de cada estação meteorológica.
plt.figure(figsize=(10, 5))
sns.barplot(data=df, x='quantidade_registros', y='acronym')
plt.xlabel('Quantidade de Registros')
plt.ylabel('Estação Meteorológica')
plt.title('Quantidade de Registros por Estação Meteorológica')
plt.show()

In [ ]:
# Plote um gráfico de barras para mostrar a quantidade de registros de cada estação por ano.
# Lembre-se de que a coluna "timestamp" contém a data no formato "YYYY-MM-DD HH:MM:SS".
# Lembre-se também que timestamp é uma palavra reservada do DuckDB, então é necessário usar aspas duplas para referenciar a coluna.
query = 'SELECT acronym,' \
        '       strftime(\'%Y\', "timestamp") AS ano,' \
        '       COUNT(*) AS quantidade_registros' \
        '  FROM dados_meteorologicos' \
        ' GROUP BY acronym, ano' \
        ' ORDER BY quantidade_registros DESC'
# Execute a query
df = con.execute(query).fetchdf()

# Plote o gráfico de barras
plt.figure(figsize=(10, 10))
sns.barplot(data=df, x='quantidade_registros', y='acronym', hue='ano')
plt.xlabel('Quantidade de Registros')
plt.ylabel('Estação Meteorológica')
plt.title('Quantidade de Registros por Estação Meteorológica e Ano')
plt.legend(title='Ano', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Plot um gráfico de linhas que mostre os valores médios de temperatura (coluna "tp_sfc") por mês para cada estação meteorológica.
# Lembre-se de que a coluna "timestamp" contém a data no formato "YYYY-MM-DD HH:MM:SS".
# Lembre-se também que timestamp é uma palavra reservada do DuckDB, então é necessário usar aspas duplas para referenciar a coluna.

# Query para calcular a média de temperatura por mês e estação meteorológica
query = 'SELECT acronym,' \
        '       strftime(\'%m\', "timestamp") AS mes,' \
        '       AVG(tp_sfc) AS media_temperatura' \
        '  FROM dados_meteorologicos' \
        ' GROUP BY acronym, mes' \
        ' ORDER BY mes'

# Execute a query
df = con.execute(query).fetchdf()

# Plote o gráfico de linhas
plt.figure(figsize=(10, 10))
sns.lineplot(data=df, x='mes', y='media_temperatura', hue='acronym', marker='o')
plt.xlabel('Mês')
plt.ylabel('Temperatura Média (°C)')
plt.title('Temperatura Média por Mês e Estação Meteorológica')
plt.legend(title='Estação Meteorológica', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()



#### 4.1. Gráfico de Boxplot

Vamos criar um gráfico de boxplot para visualizar a distribuição das temperaturas máximas e mínimas.<br>
Você irá montar uma consulta SQL para extrair as informações necessárias e gerar o gráfico.<br>
Caso a consulta esteja incorreta, a IA irá auxiliá-lo a corrigir o código. Para isso selecione o trecho de código e pressione Ctrl + I para chamar a IA.

In [ ]:
# Plot um gráfico para encontrar os outliers da coluna "tp_sfc" (temperatura do ar na superfície) para cada estação meteorológica.
# Utilize um boxplot para cada estação meteorológica apenas com os outliers.
# Lembre-se de que a coluna "timestamp" contém a data no formato "YYYY-MM-DD HH:MM:SS".
# Lembre-se também que timestamp é uma palavra reservada do DuckDB, então é necessário usar aspas duplas para referenciar a coluna.

# Query para encontrar os outliers da temperatura do ar na superfície por estação meteorológica
query = 'SELECT acronym,' \
        '       tp_sfc' \
        '  FROM dados_meteorologicos' \
        ' WHERE tp_sfc < (SELECT Q1 - 1.5 * IQR FROM (SELECT acronym,' \
        '                                                  QUANTILE(tp_sfc, 0.25) AS Q1,' \
        '                                                  QUANTILE(tp_sfc, 0.75) AS Q3,' \
        '                                                  Q3 - Q1 AS IQR' \
        '                                             FROM dados_meteorologicos' \
        '                                            GROUP BY acronyms))' \
        '    OR tp_sfc > (SELECT Q3 + 1.5 * IQR FROM (SELECT acronyms,' \
        '                                                  QUANTILE(tp_sfc, 0.25) AS Q1,' \
        '                                                  QUANTILE(tp_sfc, 0.75) AS Q3,' \
        '                                                  Q3 - Q1 AS IQR' \
        '                                             FROM dados_meteorologicos' \
        '                                            GROUP BY acronyms))'

# Execute a query
df = con.execute(query).fetchdf()


# Plote o boxplot
plt.figure(figsize=(10, 10))
sns.boxplot(data=df, x='acronym', y='tp_sfc')
plt.xlabel('Estação Meteorológica')
plt.ylabel('Temperatura do Ar na Superfície (°C)')
plt.title('Outliers da Temperatura do Ar na Superfície por Estação Meteorológica')
plt.show()

#### 5. Conclusão 🚀

Parabéns! Você concluiu o minicurso LABREN explorando dados com IA. Agora você está pronto para aplicar esses conhecimentos em seus próprios projetos. Até a próxima!